In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [2]:
%load_ext autoreload
%autoreload 2

from dynamic_adapter import load_models, get_model

In [3]:
model_name = "../llm_competition/models/mistral_7b/"
bbq_adapter_path = "../llm_competition/models/cnn_dailymail/mistral_7b/checkpoint-1000/"
cnn_adapter_path = "../llm_competition/models/cnn_dailymail/mistral_7b/checkpoint-1500/"
math_adapter_path = "../llm_competition/models/cnn_dailymail/mistral_7b/checkpoint-2000/"
routing_model_path = "./models/"

In [4]:
models = load_models(
    model_name,
    bbq_adapter_path,
    cnn_adapter_path,
    math_adapter_path,
    routing_model_path,
    device=3
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
input_string = """
'The following are multiple choice questions (with answers) about high school psychology.\n\nQuestion: Ani believes that her attitudes and behavior play a central role in what happens to her. Such a belief is likely to be associated with\nA. a strong superego.\nB. low self-esteem.\nC. low self-efficacy.\nD. an internal locus of control.\nAnswer: D\n\nQuestion: According to the Individuals with Disabilities Education Improvement Act, which of the following must an educational agency do before it changes the educational placement of a student with a disability?\nA. Give the child a trial period in the new environment\nB. Notify the parents in writing\nC. Obtain school board approval\nD. Obtain parental consent\nAnswer: B\n\nQuestion: While swimming in the ocean, Ivan is frightened by a dark shadow in the water even before he has the chance to identify what the shadow is. The synaptic connections taking place during this incident of fright are best described by which of the following?\nA. Messages are sent from the thalamus directly to the amygdala.\nB. Messages are sent from the thalamus to the "what" and "where" pathways.\nC. Messages are sent from the parasympathetic nervous system to the cerebral cortex.\nD. Messages are sent from the frontal lobes to the pituitary gland.\nAnswer: A\n\nQuestion: According to Caplan\'s model of consultee-centered case consultation, the consultant is primarily interested in\nA. identifying the causes and solutions of the client\'s presenting problems\nB. identifying and eliminating the causes of the consultee\'s difficulties in handling a problem\nC. establishing a hierarchy of authority to enable effective decision making\nD. presenting a single, well-defined and unambiguous course of action for the consultant to overcome skills deficits\nAnswer: B\n\nQuestion: Pascale is interested in the processing strategies children use to learn new information. Pascale would best be classified as what type of psychologist?\nA. sociocultural\nB. clinical\nC. cognitive\nD. behaviorist\nAnswer: C\n\nQuestion: Which of the following drugs is classified as a stimulant?\nA. alcohol\nB. nicotine\nC. heroin\nD. phencyclidine\nAnswer:'
"""

In [19]:
model = get_model(models, tokenizer, input_string, {"temperature": 0.6}, device="cuda:3")

math


In [18]:
models["routing"]()

Routing(
  (embedding_layer): Embedding(50257, 768)
  (routing_attention_layer): GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D()
      (c_proj): Conv1D()
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D()
      (c_proj): Conv1D()
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (routing_layer_1): Linear(in_features=768, out_features=384, bias=True)
  (routing_layer_2): Linear(in_features=384, out_features=192, bias=True)
  (routing_layer_3): Linear(in_features=192, out_features=4, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=-1)
  (ce_loss): CrossEntropyLoss()
)